[![Open In Colab](https://colab.research.googleusercontent.com/assets/colab-badge.svg)](https://colab.research.google.com)

Si tu repo está en GitHub, usa este enlace editando USER/REPO/BRANCH:
[Open in Colab (GitHub)](https://colab.research.google.com/github/USER/REPO/blob/BRANCH/professor/pandas_v2/06_pipe_sql.ipynb)



# 06 - Pipelines con `pipe` y consultas SQL con DuckDB

Objetivos:
- Encadenar transformaciones legibles con `pipe`.
- Ejecutar SQL sobre `DataFrame` de pandas con `duckdb`.
- Combinar ambos enfoques en un flujo práctico.


In [1]:
import pandas as pd

print("Creamos un DataFrame simple y funciones para `pipe`:")
df = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "categoria": ["A", "A", "B", "B", "B"],
    "monto": [50, -10, 100, 5, 1000],
})
print(df)

# Funciones puras que devuelven DataFrame transformado
def filtra_positivos(x: pd.DataFrame) -> pd.DataFrame:
    return x[x["monto"] > 0]

def agrega_flags(x: pd.DataFrame) -> pd.DataFrame:
    return x.assign(alto=x["monto"] >= 100)

print("\nAplicamos pipeline con pipe:")
res = (df
       .pipe(filtra_positivos)
       .pipe(agrega_flags)
      )
print(res)


Creamos un DataFrame simple y funciones para `pipe`:
   id categoria  monto
0   1         A     50
1   2         A    -10
2   3         B    100
3   4         B      5
4   5         B   1000

Aplicamos pipeline con pipe:
   id categoria  monto   alto
0   1         A     50  False
2   3         B    100   True
3   4         B      5  False
4   5         B   1000   True


## SQL con DuckDB sobre pandas

- Intentamos importar `duckdb`. Si no está, te indicamos cómo instalarlo (`pip install duckdb`).
- Ejecutamos una consulta simple para agrupar por `categoria`.


In [2]:
try:
    import duckdb  # type: ignore
    print("duckdb importado correctamente.")
except Exception as e:
    print("No se pudo importar duckdb. Si deseas usar SQL, instala con:")
    print("!pip install duckdb --quiet")
    duckdb = None

if duckdb is not None:
    print("\nConsulta SQL sobre DataFrame `res` (ya transformado con pipe):")
    q = """
    SELECT categoria, COUNT(*) AS n, SUM(monto) AS total
    FROM res
    GROUP BY 1
    ORDER BY total DESC
    """
    out = duckdb.query(q).to_df()
    print(out)


No se pudo importar duckdb. Si deseas usar SQL, instala con:
!pip install duckdb --quiet


## Combinar `pipe` + SQL: patrón de trabajo

- Usar `pipe` para dejar datos listos (limpios y enriquecidos).
- Ejecutar SQL para resumir/consultar rápidamente.
- Alternar con `pivot_table`/`groupby` según el caso/prioridad del equipo.


In [3]:
print("Pipeline -> SQL en una sola secuencia (si existe duckdb):")
if 'duckdb' in globals() and duckdb is not None:
    pipeline = (df
                .pipe(filtra_positivos)
                .pipe(agrega_flags)
               )
    q = """
    SELECT alto, COUNT(*) AS n, AVG(monto) AS promedio
    FROM pipeline
    GROUP BY 1
    ORDER BY promedio DESC
    """
    print(duckdb.query(q).to_df())
else:
    print("Instala duckdb para ejecutar SQL sobre pandas.")


Pipeline -> SQL en una sola secuencia (si existe duckdb):
Instala duckdb para ejecutar SQL sobre pandas.
